In [43]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings('ignore')

from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

from pandas_profiling import ProfileReport

sns.set(style='whitegrid', font_scale=1.5)


RANDOM_STATE=42

In [6]:
sales_data = pd.read_csv('sales_train.csv')
test_data = pd.read_csv('test.csv')
item_categories = pd.read_csv('item_categories.csv')
items = pd.read_csv('items.csv')
shops = pd.read_csv('shops.csv')

In [15]:
print(sales_data.head())
print(sales_data.info())

         date  date_block_num  shop_id  item_id  item_price  item_cnt_day
0  02.01.2013               0       59    22154      999.00           1.0
1  03.01.2013               0       25     2552      899.00           1.0
2  05.01.2013               0       25     2552      899.00          -1.0
3  06.01.2013               0       25     2554     1709.05           1.0
4  15.01.2013               0       25     2555     1099.00           1.0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 6 columns):
 #   Column          Dtype  
---  ------          -----  
 0   date            object 
 1   date_block_num  int64  
 2   shop_id         int64  
 3   item_id         int64  
 4   item_price      float64
 5   item_cnt_day    float64
dtypes: float64(2), int64(3), object(1)
memory usage: 134.4+ MB
None


In [24]:
sales_data['date'] = pd.to_datetime(sales_data['date'], format= '%d.%m.%Y')
sales_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 6 columns):
 #   Column          Dtype         
---  ------          -----         
 0   date            datetime64[ns]
 1   date_block_num  int64         
 2   shop_id         int64         
 3   item_id         int64         
 4   item_price      float64       
 5   item_cnt_day    float64       
dtypes: datetime64[ns](1), float64(2), int64(3)
memory usage: 134.4 MB


In [14]:
print(test_data.head())
print(test_data.info())

   ID  shop_id  item_id
0   0        5     5037
1   1        5     5320
2   2        5     5233
3   3        5     5232
4   4        5     5268
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214200 entries, 0 to 214199
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype
---  ------   --------------   -----
 0   ID       214200 non-null  int64
 1   shop_id  214200 non-null  int64
 2   item_id  214200 non-null  int64
dtypes: int64(3)
memory usage: 4.9 MB
None


In [16]:
print(item_categories.head())
print(item_categories.info())

        item_category_name  item_category_id
0  PC - Гарнитуры/Наушники                 0
1         Аксессуары - PS2                 1
2         Аксессуары - PS3                 2
3         Аксессуары - PS4                 3
4         Аксессуары - PSP                 4
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   item_category_name  84 non-null     object
 1   item_category_id    84 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.4+ KB
None


In [17]:
print(items.head())
print(items.info())

                                           item_name  item_id  item_category_id
0          ! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.)         D        0                40
1  !ABBYY FineReader 12 Professional Edition Full...        1                76
2      ***В ЛУЧАХ СЛАВЫ   (UNV)                    D        2                40
3    ***ГОЛУБАЯ ВОЛНА  (Univ)                      D        3                40
4        ***КОРОБКА (СТЕКЛО)                       D        4                40
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22170 entries, 0 to 22169
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   item_name         22170 non-null  object
 1   item_id           22170 non-null  int64 
 2   item_category_id  22170 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 519.7+ KB
None


In [18]:
print(shops.head())
print(shops.info())

                        shop_name  shop_id
0   !Якутск Орджоникидзе, 56 фран        0
1   !Якутск ТЦ "Центральный" фран        1
2                Адыгея ТЦ "Мега"        2
3  Балашиха ТРК "Октябрь-Киномир"        3
4        Волжский ТЦ "Волга Молл"        4
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   shop_name  60 non-null     object
 1   shop_id    60 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.1+ KB
None


In [25]:
SalesData = sales_data.copy()
SalesData = SalesData.pivot_table(
    index=['shop_id', 'item_id'],
    values=['item_cnt_day'],
    columns=['date_block_num'],
    fill_value=0,
    aggfunc='sum'
).reset_index()

In [26]:
SalesData

shop_id item_id item_cnt_day               ...                     
date_block_num                            0   1  2  3  4  ... 27 28 29 30 31 32 33
0                    0      30            0  31  0  0  0  ...  0  0  0  0  0  0  0
1                    0      31            0  11  0  0  0  ...  0  0  0  0  0  0  0
2                    0      32            6  10  0  0  0  ...  0  0  0  0  0  0  0
3                    0      33            3   3  0  0  0  ...  0  0  0  0  0  0  0
4                    0      35            1  14  0  0  0  ...  0  0  0  0  0  0  0
...                ...     ...          ...  .. .. .. ..  ... .. .. .. .. .. .. ..
424119              59   22154            1   0  0  0  0  ...  0  0  0  0  0  0  0
424120              59   22155            0   0  0  0  0  ...  0  0  0  0  0  0  0
424121              59   22162            0   0  0  0  0  ...  1  1  0  0  1  0  0
424122              59   22164            0   0  0  0  0  ...  2  0  0  1  0  0  0
424123              59   22167            0   0  0  0  0  ...  0  0  0  0  0  0  0

[424124 rows x 36 columns]

In [27]:
Final_Dataset = pd.merge(test_data,SalesData,on =['item_id','shop_id'],how = 'left')

In [28]:
Final_Dataset.head()

,ID,shop_id,item_id,"(item_cnt_day, 0)","(item_cnt_day, 1)","(item_cnt_day, 2)","(item_cnt_day, 3)","(item_cnt_day, 4)","(item_cnt_day, 5)","(item_cnt_day, 6)","(item_cnt_day, 7)","(item_cnt_day, 8)","(item_cnt_day, 9)","(item_cnt_day, 10)","(item_cnt_day, 11)","(item_cnt_day, 12)","(item_cnt_day, 13)","(item_cnt_day, 14)","(item_cnt_day, 15)","(item_cnt_day, 16)","(item_cnt_day, 17)","(item_cnt_day, 18)","(item_cnt_day, 19)","(item_cnt_day, 20)","(item_cnt_day, 21)","(item_cnt_day, 22)","(item_cnt_day, 23)","(item_cnt_day, 24)","(item_cnt_day, 25)","(item_cnt_day, 26)","(item_cnt_day, 27)","(item_cnt_day, 28)","(item_cnt_day, 29)","(item_cnt_day, 30)","(item_cnt_day, 31)","(item_cnt_day, 32)","(item_cnt_day, 33)"
0,0,5,5037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,2.0,2.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0,1.0,0.0
1,1,5,5320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,5,5233,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,0.0,1.0,3.0,1.0
3,3,5,5232,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,4,5,5268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
Final_Dataset.isnull().sum()

ID                         0
shop_id                    0
item_id                    0
(item_cnt_day, 0)     102796
(item_cnt_day, 1)     102796
(item_cnt_day, 2)     102796
(item_cnt_day, 3)     102796
(item_cnt_day, 4)     102796
(item_cnt_day, 5)     102796
(item_cnt_day, 6)     102796
(item_cnt_day, 7)     102796
(item_cnt_day, 8)     102796
(item_cnt_day, 9)     102796
(item_cnt_day, 10)    102796
(item_cnt_day, 11)    102796
(item_cnt_day, 12)    102796
(item_cnt_day, 13)    102796
(item_cnt_day, 14)    102796
(item_cnt_day, 15)    102796
(item_cnt_day, 16)    102796
(item_cnt_day, 17)    102796
(item_cnt_day, 18)    102796
(item_cnt_day, 19)    102796
(item_cnt_day, 20)    102796
(item_cnt_day, 21)    102796
(item_cnt_day, 22)    102796
(item_cnt_day, 23)    102796
(item_cnt_day, 24)    102796
(item_cnt_day, 25)    102796
(item_cnt_day, 26)    102796
(item_cnt_day, 27)    102796
(item_cnt_day, 28)    102796
(item_cnt_day, 29)    102796
(item_cnt_day, 30)    102796
(item_cnt_day,

In [32]:
Final_Dataset.fillna(0,inplace=True)
Final_Dataset.isnull().sum()

ID                    0
shop_id               0
item_id               0
(item_cnt_day, 0)     0
(item_cnt_day, 1)     0
(item_cnt_day, 2)     0
(item_cnt_day, 3)     0
(item_cnt_day, 4)     0
(item_cnt_day, 5)     0
(item_cnt_day, 6)     0
(item_cnt_day, 7)     0
(item_cnt_day, 8)     0
(item_cnt_day, 9)     0
(item_cnt_day, 10)    0
(item_cnt_day, 11)    0
(item_cnt_day, 12)    0
(item_cnt_day, 13)    0
(item_cnt_day, 14)    0
(item_cnt_day, 15)    0
(item_cnt_day, 16)    0
(item_cnt_day, 17)    0
(item_cnt_day, 18)    0
(item_cnt_day, 19)    0
(item_cnt_day, 20)    0
(item_cnt_day, 21)    0
(item_cnt_day, 22)    0
(item_cnt_day, 23)    0
(item_cnt_day, 24)    0
(item_cnt_day, 25)    0
(item_cnt_day, 26)    0
(item_cnt_day, 27)    0
(item_cnt_day, 28)    0
(item_cnt_day, 29)    0
(item_cnt_day, 30)    0
(item_cnt_day, 31)    0
(item_cnt_day, 32)    0
(item_cnt_day, 33)    0
dtype: int64

In [34]:
Final_Dataset.drop(['ID','shop_id','item_id'], axis=1,inplace=True)
Final_Dataset.head()

,"(item_cnt_day, 0)","(item_cnt_day, 1)","(item_cnt_day, 2)","(item_cnt_day, 3)","(item_cnt_day, 4)","(item_cnt_day, 5)","(item_cnt_day, 6)","(item_cnt_day, 7)","(item_cnt_day, 8)","(item_cnt_day, 9)","(item_cnt_day, 10)","(item_cnt_day, 11)","(item_cnt_day, 12)","(item_cnt_day, 13)","(item_cnt_day, 14)","(item_cnt_day, 15)","(item_cnt_day, 16)","(item_cnt_day, 17)","(item_cnt_day, 18)","(item_cnt_day, 19)","(item_cnt_day, 20)","(item_cnt_day, 21)","(item_cnt_day, 22)","(item_cnt_day, 23)","(item_cnt_day, 24)","(item_cnt_day, 25)","(item_cnt_day, 26)","(item_cnt_day, 27)","(item_cnt_day, 28)","(item_cnt_day, 29)","(item_cnt_day, 30)","(item_cnt_day, 31)","(item_cnt_day, 32)","(item_cnt_day, 33)"
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,2.0,2.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,0.0,1.0,3.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
Final_Dataset.shape

(214200, 34)

In [36]:
X_train, y_train = Final_Dataset.values[:,:-2], Final_Dataset.values[:, -2:-1].ravel()
X_valid, y_valid = Final_Dataset.values[:,1:-1], Final_Dataset.values[:, -1:].ravel()
X_test = Final_Dataset.values[:, 2:]

X_train

array([[0., 0., 0., ..., 1., 1., 3.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 2., 0., 1.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [38]:
%%time

xgb_model = XGBRegressor(
    max_depth=16,
    n_estimators=200,
    seed=RANDOM_STATE,
)

xgb_model.fit(X_train, y_train, early_stopping_rounds=10, eval_metric="rmse",
              eval_set=[(X_train, y_train), (X_valid, y_valid)], verbose=10)

[14:11:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:2.35818	validation_1-rmse:5.46382
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 10 rounds.
[10]	validation_0-rmse:1.51766	validation_1-rmse:5.07546
[20]	validation_0-rmse:1.27885	validation_1-rmse:4.9805
[30]	validation_0-rmse:1.21745	validation_1-rmse:4.95776
[40]	validation_0-rmse:1.19501	validation_1-rmse:4.93493
[50]	validation_0-rmse:1.1877	validation_1-rmse:4.92431
[60]	validation_0-rmse:1.18341	validation_1-rmse:4.9204
[70]	validation_0-rmse:1.18115	validation_1-rmse:4.91957
Stopping. Best iteration:
[64]	validation_0-rmse:1.18221	validation_1-rmse:4.91931

CPU times: user 1min 36s, sys: 54.2 ms, total: 1min 36s
Wall time: 1min 36s


In [39]:
y_pred = xgb_model.predict(X_valid)
print('XGBoost RMSE =', mean_squared_error(y_valid, y_pred, squared=False))

XGBoost RMSE = 4.920506136505432


In [40]:
tf.keras.backend.clear_session()

lstm_model = tf.keras.Sequential([
    tf.keras.layers.Reshape(input_shape=(32,), target_shape=(32, 1,)),
    tf.keras.layers.LSTM(units=32, input_shape=(32, 1)),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(1)
])

lstm_model.compile(
    loss='mse',
    optimizer=tf.keras.optimizers.Adam(0.1),
    metrics=['mse']
)

lstm_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 32, 1)             0         
_________________________________________________________________
lstm (LSTM)                  (None, 32)                4352      
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 4,385
Trainable params: 4,385
Non-trainable params: 0
_________________________________________________________________


In [41]:
%%time

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_mse', patience=10)

lstm_model.fit(X_train, y_train, batch_size=4096, epochs=30,
          validation_data=(X_valid, y_valid),
          callbacks=[early_stop])

Epoch 1/30
53/53 [==============================] - 21s 376ms/step - loss: 6.6245 - mse: 6.6245 - val_loss: 29.6180 - val_mse: 29.6180
Epoch 2/30
53/53 [==============================] - 20s 370ms/step - loss: 4.6749 - mse: 4.6749 - val_loss: 29.4580 - val_mse: 29.4580
Epoch 3/30
53/53 [==============================] - 20s 371ms/step - loss: 4.8548 - mse: 4.8548 - val_loss: 29.3409 - val_mse: 29.3409
Epoch 4/30
53/53 [==============================] - 19s 369ms/step - loss: 4.2903 - mse: 4.2903 - val_loss: 29.2091 - val_mse: 29.2091
Epoch 5/30
53/53 [==============================] - 19s 367ms/step - loss: 8.2620 - mse: 8.2620 - val_loss: 29.6356 - val_mse: 29.6356
Epoch 6/30
53/53 [==============================] - 20s 377ms/step - loss: 4.5499 - mse: 4.5499 - val_loss: 29.5125 - val_mse: 29.5125
Epoch 7/30
53/53 [==============================] - 20s 375ms/step - loss: 3.9899 - mse: 3.9899 - val_loss: 29.3880 - val_mse: 29.3880
Epoch 8/30
53/53 [==============================] - 19s

In [42]:
y_pred = lstm_model.predict(X_valid)
print('LSTM RMSE =', mean_squared_error(y_valid, y_pred, squared=False))

LSTM RMSE = 5.386443851281027


In [54]:
Submission = pd.read_csv('sample_submission.csv')

In [57]:
result_df = Submission.copy()
result_df['XGB'] = xgb_model.predict(X_test)
result_df['LSTM'] = lstm_model.predict(X_test)
result_df

,ID,item_cnt_month,XGB,LSTM
0,0,0.5,0.382106,0.599490
1,1,0.5,0.113013,0.187495
2,2,0.5,1.263642,0.334727
3,3,0.5,0.133882,0.181606
4,4,0.5,0.113013,0.187495
...,...,...,...,...
214195,214195,0.5,0.265439,0.646201
214196,214196,0.5,0.113013,0.187495
214197,214197,0.5,0.092748,0.188640
214198,214198,0.5,0.113013,0.187495


In [59]:
TARGET = 'item_cnt_month'

result_df[TARGET] = 0.05 * result_df['XGB'] + 0.9 * result_df['LSTM']

Submission = result_df[['ID', TARGET]]
Submission.to_csv(f'output.csv', index=False)
Submission

,ID,item_cnt_month
0,0,0.558646
1,1,0.174396
2,2,0.364436
3,3,0.170139
4,4,0.174396
...,...,...
214195,214195,0.594853
214196,214196,0.174396
214197,214197,0.174413
214198,214198,0.174396
